In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)
if device == 'torch':
  torch.cuda.seed_all(777)

In [4]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [7]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [9]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [31]:
class CNN(torch.nn.Module):
  
  def __init__(self):
    super(CNN, self).__init__()
    self.keep_prob = 0.5
    # L1 ImgIn shape = (?, 28, 28, 1)
    # Conv -> (?, 28, 28, 32)
    # pool -> (?, 14, 14, 32)

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
    )

    self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p=1-self.keep_prob)
    )
    self.fc2 = torch.nn.Linear(625, 10, bias=True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.layer4(out)
    out = self.fc2(out)
    return out



In [32]:
model = CNN().to(device)

In [33]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [34]:
total_batch = len(data_loader)

In [35]:
for epoch in range(training_epochs):
  avg_cost = 0
  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.18420352
[Epoch:    2] cost = 0.0512271933
[Epoch:    3] cost = 0.0369095616
[Epoch:    4] cost = 0.0282087047
[Epoch:    5] cost = 0.0217825007
[Epoch:    6] cost = 0.0194286369
[Epoch:    7] cost = 0.0162486359
[Epoch:    8] cost = 0.0137136132
[Epoch:    9] cost = 0.0118896989
[Epoch:   10] cost = 0.00967418496
[Epoch:   11] cost = 0.0105180321
[Epoch:   12] cost = 0.00900356378
[Epoch:   13] cost = 0.00816828851
[Epoch:   14] cost = 0.00806200132
[Epoch:   15] cost = 0.00694334647


In [36]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

Accuracy: 0.9775999784469604


/usr/local/lib/python3.9/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.9/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
